## 1 - Assignment1 Neural Network model



### 1.1 Packages####

In [1]:
import numpy as np
import h5py
import random

In [65]:
def vectorized_label(label):
    tmp = np.zeros((10, 1))
    tmp[label] = 1.0
    return tmp

### 1.2 Dataset ####
Show the first 10 instances and labels.

In [115]:
with h5py.File('train_128.h5','r') as H:
    data = np.copy(H['data'])
print("The first 10 instances: ")
print(data[:10],"\n")
with h5py.File('train_label.h5','r') as H:
    label = np.copy(H['label'])
print("The first 10 instances: ",label[:10],"\n")

print("The shape of instance is: ",data.shape)
print("The shape of label is: ",label.shape)

labels = np.array([vectorized_label(tmp) for tmp in label])
data = np.array([np.reshape(tmp, (128, 1)) for tmp in data])
train_data = []
for a,b in zip(labels[:48000],data[:48000]):
    train_data.append((b,a))
train_data = np.array(train_data)
print("the size of train data is: ",len(train_data))

test_data = [] 
for a,b in zip(label[48000:],data[48000:]):
    test_data.append((b,a))
test_data = np.array(test_data)

print("the size of train data is: ",len(test_data))


The first 10 instances: 
[[ -1.26502938e+02   1.63243234e+03  -1.20922145e+03 ...,   1.11837600e+02
    3.57044951e+01  -6.63192504e-01]
 [  1.40756479e+03  -4.51681446e+02  -2.59769757e+02 ...,   9.24883807e+01
   -1.46637001e+02  -7.93095667e+01]
 [ -7.25430358e+02  -1.10359763e+03   1.07005341e+02 ...,  -2.93622934e+01
    1.30540938e+01   2.86296443e+01]
 ..., 
 [  1.82574270e+03   1.67704384e+03   3.29740502e+02 ...,  -9.53786962e+01
   -5.42099830e+01  -8.22305980e+01]
 [ -1.58283168e+03  -3.88443114e+02   1.14953389e+02 ...,   5.78324789e+01
    2.17396631e+00   6.87229558e+01]
 [ -9.46185395e+02   4.13536641e+02  -5.49866946e+01 ...,   8.39081821e+01
   -8.13921414e+01   4.63882409e+01]] 

The first 10 instances:  [9 0 0 3 0 2 7 2 5 5] 

The shape of instance is:  (60000, 128)
The shape of label is:  (60000,)
the size of train data is:  48000
the size of train data is:  12000


### 1.3 - Defining the neural network structure ####

In [92]:
class Network(object):

    def __init__(self, sizes):
        """
        :param sizes: list类型，储存每层神经网络的神经元数目
                      譬如说：sizes = [2, 3, 2] 表示输入层有两个神经元、
                      隐藏层有3个神经元以及输出层有2个神经元
        """
        # 有几层神经网络
        self.num_layers = len(sizes)
        self.sizes = sizes
        # 除去输入层，随机产生每层中 y 个神经元的 biase 值（0 - 1）
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        # 随机产生每条连接线的 weight 值（0 - 1）
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """
        前向传输计算每个神经元的值
        :param a: 输入值
        :return: 计算后每个神经元的值
        """
        for b, w in zip(self.biases, self.weights):
            # 加权求和以及加上 biase
            a = sigmoid(np.dot(w, a) + b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """
        随机梯度下降
        :param training_data: 输入的训练集
        :param epochs: 迭代次数
        :param mini_batch_size: 小样本数量
        :param eta: 学习率
        :param test_data: 测试数据集
        """
        if test_data is not None: 
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            # 搅乱训练集，让其排序顺序发生变化
            random.shuffle(training_data)
            # 按照小样本数量划分训练集
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                # 根据每个小样本来更新 w 和 b，代码在下一段
                self.update_mini_batch(mini_batch, eta)
            # 输出测试每轮结束后，神经网络的准确度
            if test_data is not None:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """
        更新 w 和 b 的值
        :param mini_batch: 一部分的样本
        :param eta: 学习率
        """
        # 根据 biases 和 weights 的行列数创建对应的全部元素值为 0 的空矩阵
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            # 根据样本中的每一个输入 x 的其输出 y，计算 w 和 b 的偏导数
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            # 累加储存偏导值 delta_nabla_b 和 delta_nabla_w
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        # 更新根据累加的偏导值更新 w 和 b，这里因为用了小样本，
        # 所以 eta 要除于小样本的长度
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """
        :param x:
        :param y:
        :return:
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # 前向传输
        activation = x
        # 储存每层的神经元的值的矩阵，下面循环会 append 每层的神经元的值
        activations = [x]
        # 储存每个未经过 sigmoid 计算的神经元的值
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # 求 δ 的值
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        # 乘于前一层的输出值
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            # 从倒数第 **l** 层开始更新，**-l** 是 python 中特有的语法表示从倒数第 l 层开始计算
            # 下面这里利用 **l+1** 层的 δ 值来计算 **l** 的 δ 值
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            #print(delta.shape)
            #print(activations[-l-1].transpose().shape)
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        # 获得预测结果
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        # 返回正确识别的个数
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """
        二次损失函数
        :param output_activations:
        :param y:
        :return:
        """
        #print(output_activations.shape)
        #print(y.shape)
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """
    求 sigmoid 函数的值
    :param z:
    :return:
    """
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """
    求 sigmoid 函数的导数
    :param z:
    :return:
    """
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
net = Network([128, 30, 10])
net.SGD(train_data, 30, 20, 0.3,test_data=test_data)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:143: RuntimeWarning: overflow encountered in exp


Epoch 0: 7034 / 12000
